# Q-Learning with Taxi-V3
A Q-Learning is implemented to navigate in a city to transport passengers from point A to point B in the shortest time.

In [1]:
import numpy as np
import gym
import random

## The environment
* The environment consists on a 5x5 grid. 
* The taxi is spawned randomly in a square. 
* The passenger is spawned randomly in one of the 4 possible locations (R, B, G, Y) and whishes to go to any of the 4 possible locations too.

## The reward system
* -1 for each time step
* +20 for succesfully deliver the passenger
* -10 for illegal actions (pick up or put down the passenger at the outside of the destination)

In [2]:
env = gym.make("Taxi-v3")
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



## Q-Table initialization
In order to create the table, we need to know the action-space and the observation-space to define the number of rows (states) and columns (actions).

In [3]:
state_space = env.observation_space.n
action_space = env.action_space.n

print('There are', state_space, 'possible states')
print('There are', action_space, 'possible actions')

Q = np.zeros((state_space, action_space))

There are 500 possible states
There are 6 possible actions


## Hyperparameters

In [8]:
total_episodes =  250000        # Total number of training episodes
total_test_episodes = 100       # Total number of test episodes
max_steps = 20                  # Max steps per episode

learning_rate = 0.01            # Learning rate
gamma =  0.99                   # Discounting rate

# Exploration parameters
max_epsilon = 1.0               # Exploration probability at the start
min_epsilon = 0.001             # Mimnimum expploration probability
decay_rate =  0.01              # Exponential decay rate for exploration probability

## Epsilon-greedy policy
Epsilon-greedy is a policy that handles the exploration/explotation trade-off.

* With probability $1-\epsilon$ we do explotation: the agent selects the action with highest state-action pair value.
* With probability $\epsilon$ we do exploration: trying ramdom action.

As the training goes, the epsilon value is reduced progressively since we will need less and less exploration and more explotation

In [5]:
def epsilon_greedy_policy(Q, state, epsilon):
    if random.uniform(0,1) >  epsilon:
        action = np.argmax(Q[state])     # Explotation
    else:
        action = env.action_space.sample()    # Exploration

    return action

## Q-Learning algorithm and training

In [9]:
for episode in range(total_episodes):
    # Reset environment
    state = env.reset()
    done = False

    # Reduce epsilon
    epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate*episode)

    for step in range(max_steps):
        action = epsilon_greedy_policy(Q, state, epsilon)

        # Take the action and observe the outcome state and reward
        new_state, reward, done, info = env.step(action)

        # Update Q(s,a) <- Q(s,a) + lr [R(s,a) + gamma * max(Q(s',a') - Q(s,a))]
        Q[state][action] = Q[state][action] + learning_rate * (reward + gamma * np.max(Q[new_state] - Q[state][action]))

        if done:
            break

        state =  new_state

## Results

In [10]:
import time

rewards = []
frames = []

for episode in range(total_test_episodes):
    state =  env.reset()
    donde = False
    total_rewards = 0
    print("****************************************************")
    print('EPISODE', episode)

    for step in range(max_steps):
        env.render()

        # Take the action (index) that have the maximum expcted reward given current state
        action = np.argmax(Q[state][:])
        new_state, reward, done, info = env.step(action)
        total_rewards += reward

        if done:
            rewards.append(total_rewards)
            break

        state = new_state

env.close()
print('Score over time: ' + str(sum(rewards)/total_test_episodes))


*************
EPISODE 88
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : |_: |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : :_: |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : :_: : 